# Project3 - 多分类

本部分实现的逻辑回归的多分类，使用了鸢尾花Iris数据集，为三元分类。梯度下降方式使用的BGD，另外两种方式（SGD和MBGD）的实现在二元分类器中。

In [17]:
import numpy as np

# 用于对比sklearn结果所需的库
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [18]:
# 读取数据
np.set_printoptions(suppress=True)
old_data = np.genfromtxt("multinomialLR.csv", delimiter=',', skip_header=1)
data = np.delete(old_data,0,1)

X = data[:,:-1]
y = data[:,-1:]

# 归一化
sc = StandardScaler()
X = sc.fit_transform(X)

X = np.hstack((np.ones((X.shape[0],1)),X))

# 分割数据，其中训练集与测试集的比例为4：1
X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.2, random_state=6)

m = X_train.shape[0]

In [19]:
# 对于多分类，使用一对所有，3个类别因此y的形状为m，3
Y = np.zeros((m,3))

k = np.unique(y)
k.sort()

for cls in k.astype(int):
    Y[np.where(y_train[:,-1] == cls),cls] = 1

#print(Y)

In [20]:
# 定义参数
B = np.zeros((3,5))
alpha = 0.005
iters = 20000

In [21]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

使用BGD实现梯度下降，并使用了L2正则化。

In [22]:
def BGD(X, Y, B, alpha, iterations):
    for itreation in range(iterations):
        h = sigmoid(np.dot(X, B.T))
        loss = h - Y

        gradient = np.dot(loss.T, X) / m
        # L2正则化
        b = B[0]
        B = (1 - alpha * 0.5 / m) * B - alpha * gradient
        B[0] = b - alpha * gradient [0] / m
        # B = B - alpha * gradient

    return B

### 运行

对训练集进行训练模型，并使用测试集测试准确率。

In [23]:
# 运行
new_beta = BGD(X_train,Y,B,alpha,iters)
#print(new_beta)

# 预测
test_m = X_test.shape[0]
model_predict = sigmoid(np.dot(X_test, new_beta.T))

predict = np.zeros((test_m,1))
for i in range(model_predict.shape[0]):
    if max(model_predict[i,0],model_predict[i,1],model_predict[i,2]) == model_predict[i,0]:
        predict[i,0] = 0
    elif max(model_predict[i,0],model_predict[i,1],model_predict[i,2]) == model_predict[i,1]:
        predict[i,0] = 1
    elif max(model_predict[i,0],model_predict[i,1],model_predict[i,2]) == model_predict[i,2]:
        predict[i,0] = 2

print("我的准确率为:", np.mean(np.equal(predict,y_test)))

我的准确率为: 0.8333333333333334


### sklearn方式

调用sklearn提供的方法训练，并使用测试集测试该方法的准确率

In [24]:
# 对比sklearn
sk_data = old_data
sk_X = sk_data[:,:-1]
sk_y = sk_data[:,-1:]

sk_sc = StandardScaler()
sk_X = sk_sc.fit_transform(sk_X)

sk_X_train, sk_X_test, sk_y_train, sk_y_test =train_test_split(sk_X,sk_y,test_size=0.2, random_state=5)

lr = LogisticRegression(multi_class='ovr', penalty='l2')
lr.fit(sk_X_train,sk_y_train.reshape(m))

print("sklearn准确率为:", lr.score(sk_X_test,sk_y_test.astype('int')))


我的准确率为: 0.8333333333333334
sklearn准确率为: 1.0
